# Notebook to pipeline

## Setup

In [1]:
from pathlib import Path
import wget
import pandas as pd
import zipfile
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

## Hyperparameters

## Load data

In [19]:
target_dir = './data/'
data_url = 'https://files.consumerfinance.gov/ccdb/complaints.csv.zip'
seed = 42
sample_size = 10_000

In [3]:
file_name = wget.download(data_url, out=target_dir)

with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(target_dir)

In [21]:
df = pd.read_csv(Path().cwd() / target_dir / zip_ref.namelist()[0])

/tmp/ipykernel_76062/2130272843.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Path().cwd() / target_dir / zip_ref.namelist()[0])


## Preprocess data

In [22]:
df = df[['Product', 'Consumer complaint narrative']]
df = df.dropna()
df.columns = ['product', 'consumer_complaint_narrative']
df['category_id'] = df['product'].factorize()[0]
df = df.sample(n=sample_size, random_state=seed)

In [23]:
category_id_df = df[['product', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'product']].values)

X_train, X_test, y_train, y_test = train_test_split(df['consumer_complaint_narrative'], df['category_id'], test_size=0.2, random_state=seed)

## Define and train the model

In [25]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LinearSVC()),
])
text_clf.fit(X_train, y_train)

/home/fgi/miniconda3/envs/simple_nlp_pipeline/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('vect', CountVectorizer()), ('clf', LinearSVC())])

/home/fgi/miniconda3/envs/simple_nlp_pipeline/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('vect', CountVectorizer()), ('clf', LinearSVC())])

## Evaluate the model

In [26]:
y_pred = text_clf.predict(X_test)
report = classification_report(y_test, y_pred)

f1 = f1_score(y_test, y_pred, average='macro')

/home/fgi/miniconda3/envs/simple_nlp_pipeline/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fgi/miniconda3/envs/simple_nlp_pipeline/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fgi/miniconda3/envs/simple_nlp_pipeline/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

In [27]:
print(report)